In [1]:
# if play the code in CoLab
import pandas as pd
import seaborn as sns
from google.colab import files
import io
import warnings

warnings.filterwarnings(action='ignore')

In [2]:
# if play the code in CoLab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# if play the code in CoLab
%cd drive/My Drive/Colab Notebooks/인지프/프로젝트/ProTACT/tensorflow

/content/drive/My Drive/Colab Notebooks/인지프/프로젝트/ProTACT/tensorflow


In [4]:
# Load autoreload extension
%load_ext autoreload

# Set autoreload behavior
%autoreload 2
import pickle
import os
import time
import argparse
import random
import numpy as np
from models.ProTACT import build_ProTACT
import tensorflow as tf
from configs.configs import Configs
from utils.read_data_pr import read_pos_vocab, read_word_vocab, read_prompts_we, read_essays_prompts, read_prompts_pos
from utils.general_utils import get_scaled_down_scores, pad_hierarchical_text_sequences, get_attribute_masks, load_word_embedding_dict, build_embedd_table
from evaluators.multitask_evaluator_all_attributes import Evaluator as AllAttEvaluator
from tensorflow import keras
import matplotlib.pyplot as plt


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [5]:
# parser = argparse.ArgumentParser(description="ProTACT model")
# parser.add_argument('--test_prompt_id', type=int, default=1, help='prompt id of test essay set')
# parser.add_argument('--seed', type=int, default=12, help='set random seed')
# parser.add_argument('--model_name', type=str,
#                     choices=['ProTACT'],
#                     help='name of model')
# parser.add_argument('--num_heads', type=int, default=2, help='set the number of heads in Multihead Attention')
# parser.add_argument('--features_path', type=str, default='data/hand_crafted_v3.csv')

test_prompt_id = 1
seed = 1
num_heads = 2
features_path = '../data/hand_crafted_v3.csv'

np.random.seed(seed)
tf.random.set_seed(seed)
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)

print("Test prompt id is {} of type {}".format(test_prompt_id, type(test_prompt_id)))
print("Seed: {}".format(seed))

configs = Configs()

data_path = configs.DATA_PATH
train_path = data_path + str(test_prompt_id) + '/train.pk'
dev_path = data_path + str(test_prompt_id) + '/dev.pk'
test_path = data_path + str(test_prompt_id) + '/test.pk'
pretrained_embedding = configs.PRETRAINED_EMBEDDING
embedding_path = configs.EMBEDDING_PATH
readability_path = configs.READABILITY_PATH
prompt_path = configs.PROMPT_PATH
vocab_size = configs.VOCAB_SIZE
epochs = configs.EPOCHS
batch_size = configs.BATCH_SIZE
load = configs.LOAD

lstm_model = configs.LSTM_MODEL
print("Numhead : ", num_heads, " | Features : ", features_path, " | Pos_emb : ", configs.EMBEDDING_DIM)
print("model : ", lstm_model, " | Load : ", load )

read_configs = {
    'train_path': train_path,
    'dev_path': dev_path,
    'test_path': test_path,
    'features_path': features_path,
    'readability_path': readability_path,
    'vocab_size': vocab_size
}

Test prompt id is 1 of type <class 'int'>
Seed: 1
Numhead :  2  | Features :  ../data/hand_crafted_v3.csv  | Pos_emb :  50
model :  gru  | Load :  False


In [6]:
# read POS for prompts
pos_vocab = read_pos_vocab(read_configs)
prompt_pos_data = read_prompts_pos(prompt_path, pos_vocab) # for prompt POS embedding

# read words for prompts
word_vocab = read_word_vocab(read_configs)
prompt_data = read_prompts_we(prompt_path, word_vocab) # for prompt word embedding

# read essays and prompts
train_data, dev_data, test_data = read_essays_prompts(read_configs, prompt_data, prompt_pos_data, pos_vocab)

if pretrained_embedding:
    embedd_dict, embedd_dim, _ = load_word_embedding_dict(embedding_path)
    embedd_matrix = build_embedd_table(word_vocab, embedd_dict, embedd_dim, caseless=True)
    embed_table = [embedd_matrix]
else:
    embed_table = None

 prompt_pos size: 8
 prompt_words size: 8
 pos_x size: 9513
 readability_x size: 9513
 pos_x size: 1680
 readability_x size: 1680
 pos_x size: 1783
 readability_x size: 1783
Loading GloVe ...
OOV number =190, OOV ratio = 0.047512


In [7]:
def sample_data_dict(data_dict, sample_ratio=0.1):
    keys = list(data_dict.keys())
    # 먼저 첫 번째 키에 대한 샘플 크기를 계산합니다.
    sample_size = int(len(data_dict[keys[0]]) * sample_ratio)
    indices = np.random.choice(len(data_dict[keys[0]]), sample_size, replace=False)

    sampled_dict = {}
    for key in keys:
        if isinstance(data_dict[key], list) or isinstance(data_dict[key], np.ndarray):
            sampled_dict[key] = [data_dict[key][i] for i in indices]
        else:
            sampled_dict[key] = data_dict[key]

    return sampled_dict
# train data만 10% 사용. 나머지는 비교를 위해 그대로 사용한다.
#train_data = sample_data_dict(train_data) # train data를 10%만 사용
#dev_data = sample_data_dict(dev_data) # dev data를 10%만 사용
#test_data = sample_data_dict(test_data) # test data를 10%만 사용

In [8]:
max_sentlen = max(train_data['max_sentlen'], dev_data['max_sentlen'], test_data['max_sentlen'])
max_sentnum = max(train_data['max_sentnum'], dev_data['max_sentnum'], test_data['max_sentnum'])
prompt_max_sentlen = prompt_data['max_sentlen']
prompt_max_sentnum = prompt_data['max_sentnum']

print('max sent length: {}'.format(max_sentlen))
print('max sent num: {}'.format(max_sentnum))
print('max prompt sent length: {}'.format(prompt_max_sentlen))
print('max prompt sent num: {}'.format(prompt_max_sentnum))

train_data['y_scaled'] = get_scaled_down_scores(train_data['data_y'], train_data['prompt_ids'])
dev_data['y_scaled'] = get_scaled_down_scores(dev_data['data_y'], dev_data['prompt_ids'])
test_data['y_scaled'] = get_scaled_down_scores(test_data['data_y'], test_data['prompt_ids'])

X_train_pos = pad_hierarchical_text_sequences(train_data['pos_x'], max_sentnum, max_sentlen)
X_dev_pos = pad_hierarchical_text_sequences(dev_data['pos_x'], max_sentnum, max_sentlen)
X_test_pos = pad_hierarchical_text_sequences(test_data['pos_x'], max_sentnum, max_sentlen)

X_train_pos = X_train_pos.reshape((X_train_pos.shape[0], X_train_pos.shape[1] * X_train_pos.shape[2]))
X_dev_pos = X_dev_pos.reshape((X_dev_pos.shape[0], X_dev_pos.shape[1] * X_dev_pos.shape[2]))
X_test_pos = X_test_pos.reshape((X_test_pos.shape[0], X_test_pos.shape[1] * X_test_pos.shape[2]))

X_train_prompt = pad_hierarchical_text_sequences(train_data['prompt_words'], max_sentnum, max_sentlen)
X_dev_prompt = pad_hierarchical_text_sequences(dev_data['prompt_words'], max_sentnum, max_sentlen)
X_test_prompt = pad_hierarchical_text_sequences(test_data['prompt_words'], max_sentnum, max_sentlen)

X_train_prompt = X_train_prompt.reshape((X_train_prompt.shape[0], X_train_prompt.shape[1] * X_train_prompt.shape[2]))
X_dev_prompt = X_dev_prompt.reshape((X_dev_prompt.shape[0], X_dev_prompt.shape[1] * X_dev_prompt.shape[2]))
X_test_prompt = X_test_prompt.reshape((X_test_prompt.shape[0], X_test_prompt.shape[1] * X_test_prompt.shape[2]))

X_train_prompt_pos = pad_hierarchical_text_sequences(train_data['prompt_pos'], max_sentnum, max_sentlen)
X_dev_prompt_pos = pad_hierarchical_text_sequences(dev_data['prompt_pos'], max_sentnum, max_sentlen)
X_test_prompt_pos = pad_hierarchical_text_sequences(test_data['prompt_pos'], max_sentnum, max_sentlen)

X_train_prompt_pos = X_train_prompt_pos.reshape((X_train_prompt_pos.shape[0], X_train_prompt_pos.shape[1] * X_train_prompt_pos.shape[2]))
X_dev_prompt_pos = X_dev_prompt_pos.reshape((X_dev_prompt_pos.shape[0], X_dev_prompt_pos.shape[1] * X_dev_prompt_pos.shape[2]))
X_test_prompt_pos = X_test_prompt_pos.reshape((X_test_prompt_pos.shape[0], X_test_prompt_pos.shape[1] * X_test_prompt_pos.shape[2]))

X_train_linguistic_features = np.array(train_data['features_x'])
X_dev_linguistic_features = np.array(dev_data['features_x'])
X_test_linguistic_features = np.array(test_data['features_x'])

X_train_readability = np.array(train_data['readability_x'])
X_dev_readability = np.array(dev_data['readability_x'])
X_test_readability = np.array(test_data['readability_x'])

Y_train = np.array(train_data['y_scaled'])
Y_dev = np.array(dev_data['y_scaled'])
Y_test = np.array(test_data['y_scaled'])

X_train_attribute_rel = get_attribute_masks(Y_train)
X_dev_attribute_rel = get_attribute_masks(Y_dev)
X_test_attribute_rel = get_attribute_masks(Y_test)

print('================================')
print('X_train_pos: ', X_train_pos.shape)
print('X_train_prompt_words: ', X_train_prompt.shape)
print('X_train_prompt_pos: ', X_train_prompt_pos.shape)
print('X_train_readability: ', X_train_readability.shape)
print('X_train_ling: ', X_train_linguistic_features.shape)
print('X_train_attribute_rel: ', X_train_attribute_rel.shape)
print('Y_train: ', Y_train.shape)

print('================================')
print('X_dev_pos: ', X_dev_pos.shape)
print('X_dev_prompt_words: ', X_dev_prompt.shape)
print('X_dev_prompt_pos: ', X_dev_prompt_pos.shape)
print('X_dev_readability: ', X_dev_readability.shape)
print('X_dev_ling: ', X_dev_linguistic_features.shape)
print('X_dev_attribute_rel: ', X_dev_attribute_rel.shape)
print('Y_dev: ', Y_dev.shape)

print('================================')
print('X_test_pos: ', X_test_pos.shape)
print('X_test_prompt_words: ', X_test_prompt.shape)
print('X_test_prompt_pos: ', X_test_prompt_pos.shape)
print('X_test_readability: ', X_test_readability.shape)
print('X_test_ling: ', X_test_linguistic_features.shape)
print('X_test_attribute_rel: ', X_test_attribute_rel.shape)
print('Y_test: ', Y_test.shape)
print('================================')

max sent length: 50
max sent num: 97
max prompt sent length: 18
max prompt sent num: 8
X_train_pos:  (9513, 4850)
X_train_prompt_words:  (9513, 4850)
X_train_prompt_pos:  (9513, 4850)
X_train_readability:  (9513, 35)
X_train_ling:  (9513, 51)
X_train_attribute_rel:  (9513, 9)
Y_train:  (9513, 9)
X_dev_pos:  (1680, 4850)
X_dev_prompt_words:  (1680, 4850)
X_dev_prompt_pos:  (1680, 4850)
X_dev_readability:  (1680, 35)
X_dev_ling:  (1680, 51)
X_dev_attribute_rel:  (1680, 9)
Y_dev:  (1680, 9)
X_test_pos:  (1783, 4850)
X_test_prompt_words:  (1783, 4850)
X_test_prompt_pos:  (1783, 4850)
X_test_readability:  (1783, 35)
X_test_ling:  (1783, 51)
X_test_attribute_rel:  (1783, 9)
Y_test:  (1783, 9)


In [9]:
train_features_list = [X_train_pos, X_train_prompt, X_train_prompt_pos, X_train_linguistic_features, X_train_readability]
dev_features_list = [X_dev_pos, X_dev_prompt, X_dev_prompt_pos, X_dev_linguistic_features, X_dev_readability]
test_features_list = [X_test_pos, X_test_prompt, X_test_prompt_pos, X_test_linguistic_features, X_test_readability]

model = build_ProTACT(len(pos_vocab), len(word_vocab), max_sentnum, max_sentlen,
                  X_train_readability.shape[1],
                  X_train_linguistic_features.shape[1],
                  configs, Y_train.shape[1], num_heads, embed_table)

evaluator = AllAttEvaluator(test_prompt_id, dev_data['prompt_ids'], test_data['prompt_ids'], dev_features_list,
                            test_features_list, Y_dev, Y_test, seed)


evalutate_results = evaluator.evaluate(model, -1, print_info=True)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 prompt_word_input (InputLa  [(None, 4850)]               0         []                            
 yer)                                                                                             
                                                                                                  
 prompt_pos_input (InputLay  [(None, 4850)]               0         []                            
 er)                                                                                              
                                                                                                  
 pos_input (InputLayer)      [(None, 4850)]               0         []                            
                                                                                              

In [10]:
print(evalutate_results)

{'dev_kappa_mean': -0.007270405997827233, 'test_kappa_mean': -0.0020892790921283786, 'kappa_dev': {'score': 0.10197013343022854, 'content': 0.05945702421964416, 'organization': -0.003446119541014925, 'word_choice': -0.07809216664909968, 'sentence_fluency': -0.14467374618839557, 'conventions': -0.005219188500368732, 'prompt_adherence': 0.08902207982151655, 'language': -0.03810186811309957, 'narrativity': -0.04634980245985587}, 'kappa_test': {'score': 0.14648868837471785, 'content': 0.06041810426144245, 'organization': 0.007398595455185397, 'word_choice': -0.2034227045185928, 'sentence_fluency': -0.0013011817653225677, 'conventions': -0.022117176360200608}}


In [ ]:
# class CustomHistory(keras.callbacks.Callback):
#     def init(self):
#         self.train_loss = []
#         self.val_loss = []
#         self.train_acc = []
#         self.val_acc = []

#     def on_epoch_end(self, batch, logs={}):
#         self.train_loss.append(logs.get('loss'))
#         self.val_loss.append(logs.get('val_loss'))
#         self.train_acc.append(logs.get('acc'))
#         self.val_acc.append(logs.get('val_acc'))
# custom_hist = CustomHistory()
# custom_hist.init()

#  for ii in range(epochs):
#     print('Epoch %s/%s' % (str(ii + 1), epochs))
#     start_time = time.time()
#     model.fit(
#         train_features_list,
#         Y_train, batch_size=batch_size, epochs=5, verbose=0, shuffle=True, validation_data=(dev_features_list,Y_dev),callbacks=[custom_hist,checkpoint])
#     tt_time = time.time() - start_time
#     print("Training one epoch in %.3f s" % tt_time)
#     evaluator.evaluate(model, ii + 1)
#     print("Train Loss: ", custom_hist.train_loss[-1], "|| Val Loss: ", custom_hist.val_loss[-1])

# evaluator.print_final_info()

'''# show the loss as the graph
fig, loss_graph = plt.subplots()
loss_graph.plot(custom_hist.train_loss,'y',label='train loss')
loss_graph.plot(custom_hist.val_loss,'r',label='val loss')
loss_graph.set_xlabel('epoch')
loss_graph.set_ylabel('loss')
plt.savefig(str('images/protact/test_prompt_'+ str(test_prompt_id) + '_seed_' + str(seed) + '_loss.png'))'''

"# show the loss as the graph\nfig, loss_graph = plt.subplots()\nloss_graph.plot(custom_hist.train_loss,'y',label='train loss')\nloss_graph.plot(custom_hist.val_loss,'r',label='val loss')\nloss_graph.set_xlabel('epoch')\nloss_graph.set_ylabel('loss')\nplt.savefig(str('images/protact/test_prompt_'+ str(test_prompt_id) + '_seed_' + str(seed) + '_loss.png'))"

In [11]:
# 저장한 체크포인트 있다면: 이어서 학습 시작할때
# Checkpoint 폴더 안에 있는 .h5 파일 지울것
# model.load_weights('Checkpoint/tensor{epoch}')

checkpoint = tf.keras.callbacks.ModelCheckpoint(
    # epoch 마다 파일명 다르게 저장
    filepath='Checkpoint/bestmodel_epoch_{epoch}.h5',

    # epoch 마다 weights 들만 저장
    save_freq='epoch',
    save_weights_only = True,

    # validation accruary 가 최대일때만 저장
    monitor='val_loss',
    mode='min'
)

In [12]:
class CustomHistory(tf.keras.callbacks.Callback):
    def __init__(self, test_prompt_id, seed, save_graphs_freq=10, save_dir='training_data'):
        super(CustomHistory, self).__init__()
        self.test_prompt_id = test_prompt_id
        self.seed = seed
        self.save_graphs_freq = save_graphs_freq
        self.save_dir = save_dir
        self.train_loss = []
        self.val_loss = []
        self.train_acc = []
        self.val_acc = []
        self.epoch_times = []
        self.qwk_dev = []
        self.qwk_test = []
        self.kappa_dev_history = []
        self.kappa_test_history = []

        # Create directory if it does not exist
        os.makedirs(self.save_dir, exist_ok=True)

    def on_train_begin(self, logs=None):
        pass

    def on_epoch_begin(self, epoch, logs=None):
        self.start_time = time.time()

    def on_epoch_end(self, epoch, logs=None):
        self.train_loss.append(logs.get('loss'))
        self.val_loss.append(logs.get('val_loss'))
        self.train_acc.append(logs.get('acc'))
        self.val_acc.append(logs.get('val_acc'))
        epoch_time = time.time() - self.start_time
        self.epoch_times.append(epoch_time)
        print(f"Epoch {epoch + 1}: Train Loss: {logs.get('loss')} || Val Loss: {logs.get('val_loss')}")
        print(f"Epoch {epoch + 1} completed in {epoch_time:.3f} seconds")

        # Evaluate the model and get QWK values
        evaluation_results = evaluator.evaluate(self.model, epoch + 1)
        self.qwk_dev.append(evaluation_results["dev_kappa_mean"])
        self.qwk_test.append(evaluation_results["test_kappa_mean"])
        self.kappa_dev_history.append(evaluation_results["kappa_dev"])
        self.kappa_test_history.append(evaluation_results["kappa_test"])

        # Save data to .pkl file
        self.save_to_pickle(epoch + 1)

        # Save loss graph
        if (epoch + 1) % self.save_graphs_freq == 0 or (epoch + 1) == 100:
            self.save_loss_graph(epoch + 1)

    def save_to_pickle(self, epoch):
        data = {
            'train_loss': self.train_loss,
            'val_loss': self.val_loss,
            'train_acc': self.train_acc,
            'val_acc': self.val_acc,
            'epoch_times': self.epoch_times,
            'qwk_dev': self.qwk_dev,
            'qwk_test': self.qwk_test,
            'kappa_dev_history': self.kappa_dev_history,
            'kappa_test_history': self.kappa_test_history
        }
        filepath = os.path.join(self.save_dir, f'training_data_epoch_{epoch}.pkl')
        with open(filepath, 'wb') as f:
            pickle.dump(data, f)

    def save_loss_graph(self, epoch):
        # Plot and save the loss graph
        fig, loss_graph = plt.subplots()
        loss_graph.plot(self.train_loss, 'y', label='train loss')
        loss_graph.plot(self.val_loss, 'r', label='val loss')
        loss_graph.set_xlabel('epoch')
        loss_graph.set_ylabel('loss')
        loss_graph.set_title('Train and Validation Loss')
        loss_graph.legend()
        filepath = os.path.join(self.save_dir, f'test_prompt_{self.test_prompt_id}_seed_{self.seed}_loss_epoch_{epoch}.png')
        plt.savefig(filepath)
        plt.close()


In [13]:
custom_hist = CustomHistory(test_prompt_id=test_prompt_id, seed=seed, save_dir='training_data')

In [14]:
# checkpoint 파일들 확인

import os
import glob

# Checkpoint directory
checkpoint_dir = 'Checkpoint'
checkpoint_files = glob.glob(os.path.join(checkpoint_dir, 'bestmodel_epoch_*.h5'))

# Sort the checkpoint files by epoch number
checkpoint_files.sort(key=lambda x: int(x.split('_')[-1].split('.')[0]))

# Get the latest checkpoint file
latest_checkpoint = checkpoint_files[-1] if checkpoint_files else None
print(f'Latest checkpoint: {latest_checkpoint}')

Latest checkpoint: Checkpoint/bestmodel_epoch_50.h5


In [15]:
# Check if there is a latest checkpoint
initial_epoch = 0
load = False # True -> 학습 재개
if latest_checkpoint and load:
    print(f'Loading weights from {latest_checkpoint}')
    model.load_weights(latest_checkpoint)
    initial_epoch = int(latest_checkpoint.split('_')[-1].split('.')[0]) if latest_checkpoint else 0
else:
    print('No latest checkpoint found. Starting from scratch.')

# 모델 학습 코드
model.fit(
    train_features_list,
    Y_train,
    batch_size=batch_size,
    epochs=50, # 적절한 에포크 수로 설정
    verbose=1,
    shuffle=True,
    validation_data=(dev_features_list, Y_dev),
    callbacks=[custom_hist, checkpoint],
    initial_epoch=initial_epoch  # 학습을 재개할 에포크
)

No latest checkpoint found. Starting from scratch.
Epoch 1/50
trait num:  9
trait num:  9
952/952 [==============================] - ETA: 0s - loss: 0.0177trait num:  9
Epoch 1: Train Loss: 0.017732743173837662 || Val Loss: 0.015233942307531834
Epoch 1 completed in 255.603 seconds
56/56 [==============================] - 3s 56ms/step
CURRENT EPOCH: 1
[DEV] AVG QWK: 0.579
[DEV] score QWK: 0.708
[DEV] content QWK: 0.554
[DEV] organization QWK: 0.599
[DEV] word_choice QWK: 0.56
[DEV] sentence_fluency QWK: 0.522
[DEV] conventions QWK: 0.563
[DEV] prompt_adherence QWK: 0.559
[DEV] language QWK: 0.568
[DEV] narrativity QWK: 0.579
------------------------
[TEST] AVG QWK: 0.586
[TEST] score QWK: 0.813
[TEST] content QWK: 0.623
[TEST] organization QWK: 0.528
[TEST] word_choice QWK: 0.534
[TEST] sentence_fluency QWK: 0.527
[TEST] conventions QWK: 0.493
------------------------
[BEST TEST] AVG QWK: 0.586, {epoch}: 1
[BEST TEST] score QWK: 0.813
[BEST TEST] content QWK: 0.623
[BEST TEST] organizat

In [ ]:
# T4로 재개된 에포크(27~)
# Check if there is a latest checkpoint
initial_epoch = 0
load = True
if latest_checkpoint and load:
    print(f'Loading weights from {latest_checkpoint}')
    model.load_weights(latest_checkpoint)
    initial_epoch = int(latest_checkpoint.split('_')[-1].split('.')[0]) if latest_checkpoint else 0
else:
    print('No latest checkpoint found. Starting from scratch.')

# 모델 학습 코드
model.fit(
    train_features_list,
    Y_train,
    batch_size=batch_size,
    epochs=50, # 적절한 에포크 수로 설정
    verbose=1,
    shuffle=True,
    validation_data=(dev_features_list, Y_dev),
    callbacks=[custom_hist, checkpoint],
    initial_epoch=initial_epoch  # 학습을 재개할 에포크
)

Loading weights from Checkpoint/bestmodel_epoch_26.h5
Epoch 27/50
trait num:  9
trait num:  9
952/952 [==============================] - ETA: 0s - loss: 0.0127trait num:  9
Epoch 27: Train Loss: 0.012731757946312428 || Val Loss: 0.012677364982664585
Epoch 27 completed in 260.352 seconds
56/56 [==============================] - 3s 53ms/step
CURRENT EPOCH: 27
[DEV] AVG QWK: 0.689
[DEV] score QWK: 0.756
[DEV] content QWK: 0.665
[DEV] organization QWK: 0.736
[DEV] word_choice QWK: 0.677
[DEV] sentence_fluency QWK: 0.658
[DEV] conventions QWK: 0.745
[DEV] prompt_adherence QWK: 0.648
[DEV] language QWK: 0.653
[DEV] narrativity QWK: 0.664
------------------------
[TEST] AVG QWK: 0.649
[TEST] score QWK: 0.821
[TEST] content QWK: 0.627
[TEST] organization QWK: 0.596
[TEST] word_choice QWK: 0.641
[TEST] sentence_fluency QWK: 0.618
[TEST] conventions QWK: 0.591
------------------------
[BEST TEST] AVG QWK: 0.649, {epoch}: 27
[BEST TEST] score QWK: 0.821
[BEST TEST] content QWK: 0.627
[BEST TEST] 

In [ ]:
# CPU로 재개된 에포크(14~26)
# 일부러 학습이 완료됐던 14epoch이지만 비교를 위해 진행해보았으나,
# 값이 일부 다르게 나옴.
# Check if there is a latest checkpoint
initial_epoch = 0
load = True
if latest_checkpoint and load:
    print(f'Loading weights from {latest_checkpoint}')
    model.load_weights(latest_checkpoint)
    initial_epoch = int(latest_checkpoint.split('_')[-1].split('.')[0]) if latest_checkpoint else 0
else:
    print('No latest checkpoint found. Starting from scratch.')

# 모델 학습 코드
model.fit(
    train_features_list,
    Y_train,
    batch_size=batch_size,
    epochs=50, # 적절한 에포크 수로 설정
    verbose=1,
    shuffle=True,
    validation_data=(dev_features_list, Y_dev),
    callbacks=[custom_hist, checkpoint],
    initial_epoch=initial_epoch  # 학습을 재개할 에포크
)

Loading weights from Checkpoint/bestmodel_epoch_13.h5
Epoch 14/50
trait num:  9
trait num:  9
952/952 [==============================] - ETA: 0s - loss: 0.0130trait num:  9
Epoch 14: Train Loss: 0.013030195608735085 || Val Loss: 0.012961349450051785
Epoch 14 completed in 1676.925 seconds
56/56 [==============================] - 81s 1s/step
CURRENT EPOCH: 14
[DEV] AVG QWK: 0.679
[DEV] score QWK: 0.749
[DEV] content QWK: 0.654
[DEV] organization QWK: 0.725
[DEV] word_choice QWK: 0.664
[DEV] sentence_fluency QWK: 0.644
[DEV] conventions QWK: 0.733
[DEV] prompt_adherence QWK: 0.639
[DEV] language QWK: 0.644
[DEV] narrativity QWK: 0.655
------------------------
[TEST] AVG QWK: 0.656
[TEST] score QWK: 0.817
[TEST] content QWK: 0.645
[TEST] organization QWK: 0.618
[TEST] word_choice QWK: 0.64
[TEST] sentence_fluency QWK: 0.611
[TEST] conventions QWK: 0.606
------------------------
[BEST TEST] AVG QWK: 0.656, {epoch}: 14
[BEST TEST] score QWK: 0.817
[BEST TEST] content QWK: 0.645
[BEST TEST] o

In [ ]:
# CPU로 시작된 첫 번째 학습(~14)
# Check if there is a latest checkpoint
initial_epoch = 0
if latest_checkpoint and load:
    print(f'Loading weights from {latest_checkpoint}')
    model.load_weights(latest_checkpoint)
    initial_epoch = int(latest_checkpoint.split('_')[-1].split('.')[0]) if latest_checkpoint else 0
else:
    print('No latest checkpoint found. Starting from scratch.')

# 모델 학습 코드
model.fit(
    train_features_list,
    Y_train,
    batch_size=batch_size,
    epochs=50, # 적절한 에포크 수로 설정
    verbose=1,
    shuffle=True,
    validation_data=(dev_features_list, Y_dev),
    callbacks=[custom_hist, checkpoint],
    initial_epoch=initial_epoch  # 학습을 재개할 에포크
)

Epoch 1/50
Attention x: (None, 46, 100)
Attention x: (None, 46, 100)
Attention x: (None, None, 100)
Attention x: (None, None, 100)
Attention x: (None, None, 100)
Attention x: (None, None, 100)
Attention x: (None, None, 100)
Attention x: (None, None, 100)
Attention x: (None, None, 100)
Attention x: (None, None, 100)
Attention x: (None, None, 100)
Attention x: (None, None, 100)
Attention x: (None, None, 100)
Attention x: (None, None, 100)
Attention x: (None, None, 100)
Attention x: (None, None, 100)
Attention x: (None, None, 100)
Attention x: (None, None, 100)
Attention x: (None, None, 100)
Attention x: (None, None, 100)
Attention x: (None, None, 100)
trait num:  9
Attention x: (None, 46, 100)
Attention x: (None, 46, 100)
Attention x: (None, None, 100)
Attention x: (None, None, 100)
Attention x: (None, None, 100)
Attention x: (None, None, 100)
Attention x: (None, None, 100)
Attention x: (None, None, 100)
Attention x: (None, None, 100)
Attention x: (None, None, 100)
Attention x: (None, No

In [ ]:
# class CustomHistory(tf.keras.callbacks.Callback):
#     def on_train_begin(self, logs=None):
#         self.train_loss = []
#         self.val_loss = []
#         self.train_acc = []
#         self.val_acc = []
#         self.epoch_times = []

#     def on_epoch_begin(self, epoch, logs=None):
#         self.start_time = time.time()

#     def on_epoch_end(self, epoch, logs=None):
#         self.train_loss.append(logs.get('loss'))
#         self.val_loss.append(logs.get('val_loss'))
#         self.train_acc.append(logs.get('acc'))
#         self.val_acc.append(logs.get('val_acc'))
#         epoch_time = time.time() - self.start_time
#         self.epoch_times.append(epoch_time)
#         print(f"Epoch {epoch + 1}: Train Loss: {logs.get('loss')} || Val Loss: {logs.get('val_loss')}")
#         print(f"Epoch {epoch + 1} completed in {epoch_time:.3f} seconds")

#         # Evaluate the model (you might need to adjust this to your specific evaluation function)
#         evaluator.evaluate(self.model, epoch + 1)

# custom_hist = CustomHistory()
# model.fit(
#     train_features_list,
#     Y_train,
#     batch_size=batch_size,
#     epochs=epochs,
#     verbose=1,
#     shuffle=True,
#     validation_data=(dev_features_list, Y_dev),
#     callbacks=[custom_hist, checkpoint]
# )

Epoch 1/50
trait num:  9
trait num:  9
952/952 [==============================] - ETA: 0s - loss: 0.0163trait num:  9
Epoch 1: Train Loss: 0.016327474266290665 || Val Loss: 0.013890832662582397
Epoch 1 completed in 262.809 seconds
56/56 [==============================] - 10s 183ms/step
CURRENT EPOCH: 1
[DEV] AVG QWK: 0.626
[DEV] score QWK: 0.733
[DEV] content QWK: 0.616
[DEV] organization QWK: 0.639
[DEV] word_choice QWK: 0.541
[DEV] sentence_fluency QWK: 0.554
[DEV] conventions QWK: 0.617
[DEV] prompt_adherence QWK: 0.645
[DEV] language QWK: 0.647
[DEV] narrativity QWK: 0.641
------------------------
[TEST] AVG QWK: 0.593
[TEST] score QWK: 0.818
[TEST] content QWK: 0.6
[TEST] organization QWK: 0.536
[TEST] word_choice QWK: 0.523
[TEST] sentence_fluency QWK: 0.563
[TEST] conventions QWK: 0.517
------------------------
[BEST TEST] AVG QWK: 0.593, {epoch}: 1
[BEST TEST] score QWK: 0.818
[BEST TEST] content QWK: 0.6
[BEST TEST] organization QWK: 0.536
[BEST TEST] word_choice QWK: 0.523
[B

KeyboardInterrupt: 

In [ ]:
# 실행 X
# TEST: 위에서 진행된곳 까지 결과가 같다.
model.load_weights('Checkpoint/bestmodel1.h5')
evaluator.evaluate(model,1)

56/56 [==============================] - 10s 185ms/step
CURRENT EPOCH: 1
[DEV] AVG QWK: 0.626
[DEV] score QWK: 0.733
[DEV] content QWK: 0.616
[DEV] organization QWK: 0.639
[DEV] word_choice QWK: 0.541
[DEV] sentence_fluency QWK: 0.554
[DEV] conventions QWK: 0.617
[DEV] prompt_adherence QWK: 0.645
[DEV] language QWK: 0.647
[DEV] narrativity QWK: 0.641
------------------------
[TEST] AVG QWK: 0.593
[TEST] score QWK: 0.818
[TEST] content QWK: 0.6
[TEST] organization QWK: 0.536
[TEST] word_choice QWK: 0.523
[TEST] sentence_fluency QWK: 0.563
[TEST] conventions QWK: 0.517
------------------------
[BEST TEST] AVG QWK: 0.593, {epoch}: 1
[BEST TEST] score QWK: 0.818
[BEST TEST] content QWK: 0.6
[BEST TEST] organization QWK: 0.536
[BEST TEST] word_choice QWK: 0.523
[BEST TEST] sentence_fluency QWK: 0.563
[BEST TEST] conventions QWK: 0.517
--------------------------------------------------------------------------------------------------------------------------
